In [34]:
import pandas as pd
# from qipy import q
import os
import numpy as np
import time
from concurrent.futures import ThreadPoolExecutor
from multiprocessing import Manager
from joblib import Parallel, delayed
import polars as pl
import datetime as dt
import time

## 参数设置

In [35]:
# df = pl.read_csv('csp_all.csv')
# df.filter((pl.col('date')=='2025-02-13')&(pl.col('code')=='002401.SZ'))

In [36]:
# 参数设置
window_for_vol_mean = 20  # 成交额均值计算的滚动窗口
start_date = '2011-01-01'
end_date = '2025-02-18'
n_days = 756   # 过去756个交易日，约3年
lag_days = 20  # 不使用最近20个交易日的数据
min_samples = 20  # 历史窗口中要求至少252个样本，否则 IR 为空
output_folder = 'final_results'
os.makedirs(output_folder, exist_ok=True)
stock_codes = ['*']  # 全部股票
num1 = 1
num2 = 11



In [37]:
df_price = (
    pl.read_csv('data.csv')
    .select(['date', 'code', 'open_adj', 'high_adj', 'low_adj', 'close_adj', 'amt'])
    .filter(~pl.col('code').str.contains('HK'))
    .filter(~pl.col('code').str.contains('BJ'))
    .with_columns(
        pl.col('code').cast(pl.Categorical),
    # .drop_nulls()
        # ((pl.col('close_adj').shift(-lag_days) / pl.col('close_adj') - 1).over('code')).alias('future20_return'),
        ((pl.col('close_adj').shift(20) / (pl.col('close_adj').shift(252)) - 1)).over('code').alias('Mom'),
    ).filter(pl.col("amt") != 0)#.drop_nulls('Mom')
)

df_price

date,code,open_adj,high_adj,low_adj,close_adj,amt,Mom
str,cat,f64,f64,f64,f64,f64,f64
"""2010-01-04""","""000001.SZ""",880.403743,882.558075,850.243092,851.320259,5.8025e8,null
"""2010-01-05""","""000001.SZ""",852.75648,858.14231,816.850944,836.598989,1.2935e9,null
"""2010-01-06""","""000001.SZ""",834.803712,834.803712,815.773778,822.236774,9.4445e8,null
"""2010-01-07""","""000001.SZ""",822.236774,827.622605,804.284006,813.26039,8.0417e8,null
"""2010-01-08""","""000001.SZ""",807.87456,816.850944,802.48873,811.465114,6.5067e8,null
…,…,…,…,…,…,…,…
"""2025-02-07""","""689009.SH""",52.607979,52.849669,50.755018,51.238399,6.81044124e8,0.754396
"""2025-02-10""","""689009.SH""",50.855722,52.245443,50.603961,50.654313,4.84185058e8,0.877429
"""2025-02-11""","""689009.SH""",50.674454,51.208187,50.110509,50.614031,3.62553983e8,0.925422


In [38]:
df_price.filter(df_price.is_duplicated())

date,code,open_adj,high_adj,low_adj,close_adj,amt,Mom
str,cat,f64,f64,f64,f64,f64,f64


In [39]:
df_000985 = df_price.filter(pl.col('code') == '000985.SZ').select(['date','Mom'])
df_000985

date,Mom
str,f64
"""2010-01-04""",null
"""2010-01-05""",null
"""2010-01-06""",null
"""2010-01-07""",null
"""2010-01-08""",null
…,…
"""2025-02-07""",0.123794
"""2025-02-10""",0.066004
"""2025-02-11""",0.0868


In [40]:
# 计算每日其余股票对000985.SZ的超额收益
df_price = df_price.join(df_000985, on='date', suffix="_target") \
    .with_columns((pl.col('Mom') - pl.col('Mom_target')/(pl.col('Mom_target')+1)).alias('Mom'))

In [41]:
dates = df_price['date'].unique().to_list()
date_list = pd.to_datetime(dates)
date_list

DatetimeIndex(['2017-06-14', '2023-03-17', '2018-11-05', '2012-12-17',
               '2022-06-02', '2020-07-15', '2017-04-26', '2021-05-24',
               '2012-04-17', '2010-09-17',
               ...
               '2011-06-10', '2020-07-03', '2016-11-30', '2013-05-16',
               '2019-06-13', '2023-09-25', '2021-02-05', '2014-07-28',
               '2018-11-15', '2019-11-27'],
              dtype='datetime64[ns]', length=3661, freq=None)

In [42]:
null_counts = df_price.select([pl.col(col).is_null().sum().alias(col) for col in df_price.columns])

null_counts

date,code,open_adj,high_adj,low_adj,close_adj,amt,Mom,Mom_target
u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,1297973,449496


- <span style="color: yellow;">**groupby**</span> -> 标记类聚的分组，返回一个跟组的个数一致的 DataFrame

- <span style="color: yellow;">**over**</span> -> 标记我们希望对这个分组进行计算，但是不会更改原有 DataFrame 的形状

In [43]:
def aggregate_k_lines(df_group,i):
        result = df_price.with_columns(
            k_high=pl.col("high_adj").rolling_max(window_size=i).over("code"),
            k_low=pl.col("low_adj").rolling_min(window_size=i).over("code"),
            k_open=pl.col("open_adj").shift(i-1).over("code"),
            k_close=pl.col("close_adj"),
            amount_mean=pl.col("amt").rolling_mean(window_size=window_for_vol_mean).over("code"),

        )
        # result = result.with_columns(
        #     pl.col("k_amount").rolling_mean(window_size=window_for_vol_mean).over("code").alias("amount_mean")
        # )
        return result.select(['date','code','k_high','k_low','k_open','k_close','amt','amount_mean','Mom'])
df_k_line = aggregate_k_lines(df_price,5)
df_k_line

date,code,k_high,k_low,k_open,k_close,amt,amount_mean,Mom
str,cat,f64,f64,f64,f64,f64,f64,f64
"""2010-01-04""","""000001.SZ""",null,null,null,851.320259,5.8025e8,null,null
"""2010-01-05""","""000001.SZ""",null,null,null,836.598989,1.2935e9,null,null
"""2010-01-06""","""000001.SZ""",null,null,null,822.236774,9.4445e8,null,null
"""2010-01-07""","""000001.SZ""",null,null,null,813.26039,8.0417e8,null,null
"""2010-01-08""","""000001.SZ""",882.558075,802.48873,880.403743,811.465114,6.5067e8,null,null
…,…,…,…,…,…,…,…,…
"""2025-02-07""","""689009.SH""",54.853687,49.20417,53.373332,51.238399,6.81044124e8,5.3488e8,0.644239
"""2025-02-10""","""689009.SH""",53.161853,49.20417,51.329033,50.654313,4.84185058e8,5.2103e8,0.815511
"""2025-02-11""","""689009.SH""",53.161853,49.20417,52.064175,50.614031,3.62553983e8,4.9237e8,0.845555


In [44]:
def calculate_price_indicators(df_k_line):
        df_k_line = df_k_line.with_columns(
            pl.col("k_close").shift(1).over("code").alias("pre_close")
        )

        # 第二步：使用已存在的列计算其他指标
        df_k_line = df_k_line.with_columns(
            ((pl.col("k_open") - pl.col("k_close")).abs() / pl.col("pre_close")).alias("body"),
            ((pl.col("k_high") - pl.max_horizontal("k_open", "k_close")) / pl.col("pre_close")).alias("upper_shadow"),
            ((pl.min_horizontal("k_open", "k_close") - pl.col("k_low")) / pl.col("pre_close")).alias("lower_shadow"),
            ((pl.col("amt")) / pl.col("amount_mean").over("code")).alias("amt_ratio"),
        ).drop_nulls(subset=["lower_shadow", 'upper_shadow', 'body', 'amt_ratio'])

        # 打上标记
        df_k_line = df_k_line.with_columns(
            (pl.when(pl.col("k_close") > pl.col("k_open")).then(pl.lit("阳")).otherwise(pl.lit("阴")) +
             pl.when(pl.col("body") <= 0.015).then(pl.lit("小"))
             .when((pl.col("body") > 0.015) & (pl.col("body") <= 0.03)).then(pl.lit("中"))
             .otherwise(pl.lit("大")) +
             pl.when(pl.col("upper_shadow") > 0.02).then(pl.lit("长上")).otherwise(pl.lit("短上")) +
             pl.when(pl.col("lower_shadow") > 0.02).then(pl.lit("长下")).otherwise(pl.lit("短下"))).alias("price_pattern")
        )
        df_k_line = df_k_line.with_columns(
            pattern=(pl.when(pl.col("amt_ratio") >= 1.2).then(pl.lit("放")).when(pl.col("amt_ratio") <= 0.8).then(pl.lit("缩")).otherwise(pl.lit("常")) +
                     # 前两日和今日的price_pattern
                     pl.col("price_pattern").shift(2) + "_" + pl.col("price_pattern").shift(1) + "_" + pl.col("price_pattern")).cast(pl.Categorical)
        ).drop_nulls(subset=['pattern'])
        # print(df_k_line)
        return df_k_line.select(['date','code','pattern','Mom','k_close'])
df_k_line = calculate_price_indicators(df_k_line)
df_k_line

date,code,pattern,Mom,k_close
str,cat,cat,f64,f64
"""2010-02-02""","""000001.SZ""","""缩阴大长上长下_阴大短上短下_阴大短上短下""",null,767.301304
"""2010-02-03""","""000001.SZ""","""常阴大短上短下_阴大短上短下_阳大短上长下""",null,810.747003
"""2010-02-04""","""000001.SZ""","""缩阴大短上短下_阳大短上长下_阳中短上长下""",null,797.102899
"""2010-02-05""","""000001.SZ""","""缩阳大短上长下_阳中短上长下_阳小长上长下""",null,789.921792
"""2010-02-08""","""000001.SZ""","""缩阳中短上长下_阳小长上长下_阳小长上短下""",null,774.123356
…,…,…,…,…
"""2025-02-07""","""689009.SH""","""放阴大长上长下_阴大短上长下_阴大长上长下""",0.644239,51.238399
"""2025-02-10""","""689009.SH""","""常阴大短上长下_阴大长上长下_阴小长上长下""",0.815511,50.654313
"""2025-02-11""","""689009.SH""","""缩阴大长上长下_阴小长上长下_阴中长上长下""",0.845555,50.614031


In [45]:
df_k_line.filter(pl.col('pattern')==pl.lit('缩阴大短上长下_阴中长上长下_阴大短上长下'))

date,code,pattern,Mom,k_close
str,cat,cat,f64,f64
"""2022-09-01""","""000004.SZ""","""缩阴大短上长下_阴中长上长下_阴大短上长下""",-0.794101,37.631362
"""2010-06-03""","""000006.SZ""","""缩阴大短上长下_阴中长上长下_阴大短上长下""",null,90.460654
"""2024-06-12""","""000006.SZ""","""缩阴大短上长下_阴中长上长下_阴大短上长下""",-0.111802,137.896672
"""2012-07-16""","""000008.SZ""","""缩阴大短上长下_阴中长上长下_阴大短上长下""",0.006691,26.841225
"""2013-07-25""","""000009.SZ""","""缩阴大短上长下_阴中长上长下_阴大短上长下""",0.319172,45.493477
…,…,…,…,…
"""2022-09-21""","""688722.SH""","""缩阴大短上长下_阴中长上长下_阴大短上长下""",null,18.87
"""2022-12-05""","""688786.SH""","""缩阴大短上长下_阴中长上长下_阴大短上长下""",-0.388184,47.431475
"""2022-12-29""","""688789.SH""","""缩阴大短上长下_阴中长上长下_阴大短上长下""",-0.702963,163.642226


动量因子为：

$$
Mom = \frac{}{}
$$

先按照csp中的pattern分类算IC、IR(保证每日至少有十支股票才触发)，之后筛选出总有效天数超过十天的形态

In [46]:
n = 2
IC = df_k_line.with_columns(
    (pl.col('k_close').shift(2)/pl.col('k_close').shift(1)-1).over('code')
).drop_nulls()
IC = IC.group_by(['date','pattern']).agg(
    corr =pl.corr('k_close','Mom'),
    spr_corr =pl.corr('k_close','Mom', method="spearman"),
    count = pl.col('Mom').count(),
).sort('date').drop_nulls().drop_nans().filter(
    pl.col('count')>10
)
print(f"corr:{IC['corr'].mean()},spr_corr:{IC['spr_corr'].mean()}")
IC


corr:-0.0006449272652282702,spr_corr:-0.0013934503461657173


date,pattern,corr,spr_corr,count
str,cat,f64,f64,u32
"""2011-01-18""","""缩阴小长上长下_阴中长上短下_阴大长上短下""",0.777428,0.631868,13
"""2011-01-18""","""缩阴大长上短下_阴大长上短下_阴大长上长下""",0.264169,0.194737,19
"""2011-01-18""","""缩阴大短上短下_阴大长上短下_阴大短上短下""",-0.011379,0.032083,58
"""2011-01-18""","""缩阴大短上长下_阴大长上短下_阴大长上短下""",-0.571892,-0.450549,13
"""2011-01-18""","""缩阴大短上长下_阴大短上短下_阴大长上短下""",0.510553,0.475524,12
…,…,…,…,…
"""2025-02-13""","""常阴小短上短下_阳小短上短下_阴小短上短下""",0.086581,0.066234,21
"""2025-02-13""","""常阳小短上长下_阳中短上短下_阳小短上短下""",-0.030575,-0.135714,15
"""2025-02-13""","""常阳大短上短下_阳大短上短下_阳大短上短下""",0.067985,0.114092,22


In [52]:
IC.filter(pl.col("pattern")==pl.lit("常阳大长上短下_阳小长上短下_阴大短上长下"	))

date,pattern,corr,spr_corr,count
str,cat,f64,f64,u32
"""2024-10-14""","""常阳大长上短下_阳小长上短下_阴大短上长下""",-0.285053,-0.307781,133


In [48]:
target = IC.group_by(['pattern']).agg(
    IC = pl.col('corr').mean(),
    ICIR = pl.col('corr').mean()/pl.col('corr').std(),
    RankIC = pl.col('spr_corr').mean(),
    RankICIR = pl.col('spr_corr').mean()/pl.col('spr_corr').std(),
    num_count = pl.col("count").sum(),
    date_count = pl.col("count").count(),
    last_date = pl.col('date').last(),
    date_list = pl.col('date')
).sort('IC').filter(pl.col('num_count')>100).filter(pl.col('date_count')>10)#.tail(20)
target

pattern,IC,ICIR,RankIC,RankICIR,num_count,date_count,last_date,date_list
cat,f64,f64,f64,f64,u32,u32,str,list[str]
"""常阳大长上短下_阳大短上短下_阳大短上长下""",-0.255168,-1.42275,-0.259051,-1.210524,391,13,"""2024-10-28""","[""2012-03-09"", ""2015-08-17"", … ""2024-10-28""]"
"""常阳中长上短下_阴小长上短下_阴大长上短下""",-0.24557,-0.78069,-0.22523,-0.721194,203,13,"""2024-06-24""","[""2011-05-23"", ""2012-09-17"", … ""2024-06-24""]"
"""缩阴大短上长下_阳大短上短下_阳大短上短下""",-0.24163,-0.848685,-0.183293,-0.804873,285,14,"""2022-11-07""","[""2011-06-07"", ""2011-08-26"", … ""2022-11-07""]"
"""放阳大长上长下_阳大短上短下_阳大短上长下""",-0.234003,-0.934878,-0.21516,-0.963023,215,13,"""2024-12-02""","[""2013-07-03"", ""2015-05-26"", … ""2024-12-02""]"
"""缩阴小短上短下_阴中短上短下_阳小短上短下""",-0.228222,-0.74858,-0.175838,-0.497826,216,13,"""2024-05-22""","[""2016-12-09"", ""2017-03-06"", … ""2024-05-22""]"
…,…,…,…,…,…,…,…,…
"""缩阴大短上长下_阴大短上短下_阳小长上短下""",0.236767,0.818255,0.280502,1.054515,247,13,"""2024-10-16""","[""2011-08-01"", ""2011-12-12"", … ""2024-10-16""]"
"""缩阳小长上长下_阴大长上短下_阴大短上短下""",0.237685,0.871109,0.208032,0.791619,253,14,"""2024-12-16""","[""2012-06-05"", ""2013-04-26"", … ""2024-12-16""]"
"""缩阴大短上长下_阴中长上短下_阴大长上短下""",0.238334,1.004837,0.225068,1.898774,236,11,"""2024-12-31""","[""2011-01-25"", ""2011-06-17"", … ""2024-12-31""]"


In [49]:
target.head(10)

pattern,IC,ICIR,RankIC,RankICIR,num_count,date_count,last_date,date_list
cat,f64,f64,f64,f64,u32,u32,str,list[str]
"""常阳大长上短下_阳大短上短下_阳大短上长下""",-0.255168,-1.42275,-0.259051,-1.210524,391,13,"""2024-10-28""","[""2012-03-09"", ""2015-08-17"", … ""2024-10-28""]"
"""常阳中长上短下_阴小长上短下_阴大长上短下""",-0.24557,-0.78069,-0.22523,-0.721194,203,13,"""2024-06-24""","[""2011-05-23"", ""2012-09-17"", … ""2024-06-24""]"
"""缩阴大短上长下_阳大短上短下_阳大短上短下""",-0.24163,-0.848685,-0.183293,-0.804873,285,14,"""2022-11-07""","[""2011-06-07"", ""2011-08-26"", … ""2022-11-07""]"
"""放阳大长上长下_阳大短上短下_阳大短上长下""",-0.234003,-0.934878,-0.21516,-0.963023,215,13,"""2024-12-02""","[""2013-07-03"", ""2015-05-26"", … ""2024-12-02""]"
"""缩阴小短上短下_阴中短上短下_阳小短上短下""",-0.228222,-0.74858,-0.175838,-0.497826,216,13,"""2024-05-22""","[""2016-12-09"", ""2017-03-06"", … ""2024-05-22""]"
"""缩阳大长上短下_阳大长上长下_阳大长上长下""",-0.225761,-1.011233,-0.186967,-0.987749,321,12,"""2024-04-25""","[""2015-07-17"", ""2015-09-09"", … ""2024-04-25""]"
"""常阳大长上短下_阳大长上短下_阳大短上长下""",-0.214971,-0.701138,-0.124666,-0.434716,406,18,"""2024-10-28""","[""2014-02-17"", ""2015-04-10"", … ""2024-10-28""]"
"""缩阴大短上长下_阴小长上长下_阴小短上长下""",-0.212188,-0.633092,-0.12017,-0.414298,242,15,"""2025-01-09""","[""2011-01-28"", ""2013-07-02"", … ""2025-01-09""]"
"""放阴小长上长下_阳大短上长下_阳大长上长下""",-0.209564,-0.760808,-0.156513,-0.593858,309,19,"""2025-01-15""","[""2015-04-13"", ""2015-05-12"", … ""2025-01-15""]"


In [50]:
target.tail(10)

pattern,IC,ICIR,RankIC,RankICIR,num_count,date_count,last_date,date_list
cat,f64,f64,f64,f64,u32,u32,str,list[str]
"""缩阳小长上短下_阳小长上短下_阴大长上短下""",0.213921,0.73527,0.176849,0.639108,155,11,"""2024-11-01""","[""2011-11-21"", ""2012-11-08"", … ""2024-11-01""]"
"""缩阴中长上长下_阴大长上长下_阴大短上短下""",0.217977,0.711237,0.142847,0.477171,242,14,"""2024-08-08""","[""2011-12-27"", ""2015-04-20"", … ""2024-08-08""]"
"""放阴大长上长下_阴大长上短下_阴大长上长下""",0.225032,0.873482,0.148514,0.584876,221,11,"""2024-04-17""","[""2013-06-25"", ""2015-07-08"", … ""2024-04-17""]"
"""缩阴中短上长下_阴中短上短下_阴小短上短下""",0.225644,1.010188,0.172554,0.892912,197,13,"""2024-08-28""","[""2011-09-09"", ""2012-11-23"", … ""2024-08-28""]"
"""缩阳大长上短下_阴小长上长下_阴大长上短下""",0.231368,0.92838,0.204403,0.787426,177,11,"""2025-01-22""","[""2015-07-27"", ""2015-09-29"", … ""2025-01-22""]"
"""缩阴大短上长下_阴大短上短下_阳小长上短下""",0.236767,0.818255,0.280502,1.054515,247,13,"""2024-10-16""","[""2011-08-01"", ""2011-12-12"", … ""2024-10-16""]"
"""缩阳小长上长下_阴大长上短下_阴大短上短下""",0.237685,0.871109,0.208032,0.791619,253,14,"""2024-12-16""","[""2012-06-05"", ""2013-04-26"", … ""2024-12-16""]"
"""缩阴大短上长下_阴中长上短下_阴大长上短下""",0.238334,1.004837,0.225068,1.898774,236,11,"""2024-12-31""","[""2011-01-25"", ""2011-06-17"", … ""2024-12-31""]"
"""缩阴小长上长下_阴中长上短下_阴大长上短下""",0.254842,0.942643,0.219922,0.999109,211,15,"""2024-12-18""","[""2011-01-18"", ""2011-12-05"", … ""2024-12-18""]"


In [51]:
target.sort('RankIC')

pattern,IC,ICIR,RankIC,RankICIR,num_count,date_count,last_date,date_list
cat,f64,f64,f64,f64,u32,u32,str,list[str]
"""常阳大长上短下_阳大短上短下_阳大短上长下""",-0.255168,-1.42275,-0.259051,-1.210524,391,13,"""2024-10-28""","[""2012-03-09"", ""2015-08-17"", … ""2024-10-28""]"
"""常阳中长上短下_阴小长上短下_阴大长上短下""",-0.24557,-0.78069,-0.22523,-0.721194,203,13,"""2024-06-24""","[""2011-05-23"", ""2012-09-17"", … ""2024-06-24""]"
"""放阳大长上长下_阳大短上短下_阳大短上长下""",-0.234003,-0.934878,-0.21516,-0.963023,215,13,"""2024-12-02""","[""2013-07-03"", ""2015-05-26"", … ""2024-12-02""]"
"""放阴小长上短下_阴大短上短下_阴大短上短下""",-0.11611,-0.41026,-0.213415,-0.936274,184,11,"""2024-01-09""","[""2011-10-20"", ""2017-11-17"", … ""2024-01-09""]"
"""常阴小长上短下_阴中短上短下_阴中短上短下""",-0.187558,-0.600731,-0.207476,-0.738963,167,12,"""2025-01-27""","[""2016-11-01"", ""2017-02-17"", … ""2025-01-27""]"
…,…,…,…,…,…,…,…,…
"""缩阳小长上长下_阴大长上短下_阴大短上短下""",0.237685,0.871109,0.208032,0.791619,253,14,"""2024-12-16""","[""2012-06-05"", ""2013-04-26"", … ""2024-12-16""]"
"""缩阴小长上长下_阴中长上短下_阴大长上短下""",0.254842,0.942643,0.219922,0.999109,211,15,"""2024-12-18""","[""2011-01-18"", ""2011-12-05"", … ""2024-12-18""]"
"""缩阴大短上长下_阴中长上短下_阴大长上短下""",0.238334,1.004837,0.225068,1.898774,236,11,"""2024-12-31""","[""2011-01-25"", ""2011-06-17"", … ""2024-12-31""]"


In [ ]:
target.write_parquet('csp_mom.parquet')

In [117]:
pd.read_parquet('csp_mom.parquet')

,pattern,IC,ICIR,RankIC,RankICIR,num_count,date_count,last_date,date_list
0,缩阳中短上短下_阴小长上短下_阴中短上短下,-0.197083,-0.723026,-0.129123,-0.360028,179,12,2024-12-16,"[2012-05-10, 2013-02-22, 2013-03-27, 2017-02-1..."
1,缩阴小短上长下_阴小短上短下_阳小短上短下,-0.122039,-0.476782,-0.102250,-0.517671,344,20,2024-03-20,"[2012-06-15, 2012-11-23, 2016-05-25, 2016-08-0..."
2,缩阳小短上短下_阳小短上短下_阴中短上短下,-0.111049,-0.492641,-0.173093,-0.656871,251,15,2024-03-22,"[2012-11-08, 2012-11-26, 2016-08-11, 2016-09-2..."
3,缩阳中短上短下_阴小短上短下_阳小短上短下,-0.106073,-0.315453,-0.119460,-0.347189,267,19,2024-03-20,"[2013-02-20, 2014-08-08, 2016-08-12, 2016-12-2..."
4,常阴大短上短下_阴中短上短下_阳小短上长下,-0.083874,-0.235436,0.028548,0.089302,169,12,2024-09-19,"[2013-11-15, 2018-12-03, 2019-06-11, 2019-11-1..."
...,...,...,...,...,...,...,...,...,...
1314,缩阴大长上长下_阴大长上长下_阳中长上长下,0.307251,0.916411,0.370480,1.582607,212,12,2024-11-29,"[2014-03-04, 2015-08-03, 2015-08-04, 2015-08-0..."
1315,缩阴大短上长下_阴中长上长下_阴大短上长下,0.315099,0.915652,0.286144,1.181263,523,24,2025-01-09,"[2011-12-26, 2013-04-10, 2013-06-28, 2013-10-3..."
1316,缩阳中长上长下_阴大长上短下_阴大长上长下,0.333767,1.265744,0.227530,0.865894,466,19,2024-11-27,"[2013-06-21, 2015-06-17, 2015-06-23, 2015-07-2..."
1317,缩阴大长上长下_阴中长上长下_阴大长上长下,0.335099,1.129898,0.287800,1.225910,298,15,2022-03-18,"[2011-12-26, 2015-04-17, 2015-05-04, 2015-06-2..."


缩阳中短上短下_阴小长上短下_阴中短上短下
缩阳大短上短下_阳中长上长下_阳大长上短下

In [22]:
# a = df_k_line.group_by(['pattern', 'date']).agg(
#                 pl.len().alias('count'),
#                 pl.sum('future20_return').alias('sum'),
#                 (pl.col('future20_return') ** 2).sum().alias('sumsq')
#             ).sort("date", descending=False).with_columns(
#                 pl.col('count').shift(lag_days).over('pattern'),
#                 pl.col('sum').shift(lag_days).over('pattern'),
#                 pl.col('sumsq').shift(lag_days).over('pattern')
#             )
# a

In [23]:
  #138_253_962

In [24]:
def calculate_ir_metrics(df_k_line, lag_days, n_days, min_samples):
        q = (
            df_k_line.lazy().group_by(['pattern', 'date']).agg(
                pl.len().alias('count'),
                pl.sum('future20_return').alias('sum'),
                (pl.col('future20_return') ** 2).sum().alias('sumsq')
            )
            .sort("date", descending=False)
            )
        date = df_k_line.select('date').unique().sort('date')
        pattern = df_k_line.select('pattern').unique().sort('pattern')
        ir_all = pattern.join(date, how="cross").lazy()

        q = ir_all.join(q, on=["pattern", "date"], how="left").fill_null(0)
        q = q.with_columns(
                pl.col('count').shift(lag_days).over('pattern'),
                pl.col('sum').shift(lag_days).over('pattern'),
                pl.col('sumsq').shift(lag_days).over('pattern')
        )#.drop_nulls(subset=['count'])
        
        q = q.with_columns(
            rolling_sum=pl.col('sum').rolling_sum(window_size=n_days, min_periods=1),
            rolling_count=pl.col('count').rolling_sum(window_size=n_days, min_periods=1),
            rolling_sumsq=pl.col('sumsq').rolling_sum(window_size=n_days, min_periods=1)
        ).sort('date', descending=False)
        
        q = q.with_columns(
            ir=pl.col('rolling_sum') / pl.col('rolling_count') /
            ((pl.col('rolling_sumsq') / pl.col('rolling_count') - pl.col('rolling_sum') / pl.col('rolling_count') ** 2) ** 0.5)
        )
        
        q = q.with_columns(
            pl.when(pl.col('rolling_count') <= min_samples).then(None).otherwise(pl.col('ir')).alias('ir')
        ).select(['date', 'pattern', 'ir']).sort(['date',], descending=False)

        return q.collect()
# ir = calculate_ir_metrics(df_k_line, lag_days, n_days, min_samples)
# ir

In [25]:
# ir.filter(pl.col('ir').is_not_null())

In [26]:
# ir.select([pl.col(col).is_null().sum().alias(col) for col in ir.columns])

In [27]:
# merged_df = ir.join(
#     df_k_line.select(['date','pattern','code']),
#     on=["date", "pattern"],
#     how="inner"
# )#.select( "date", "code","ir")
# merged_df = merged_df.with_columns(
#     pl.col('ir').rolling_mean(20, min_periods=5).over('code').alias(f'factor_{5}') 
# ).drop_nulls(subset=[f'factor_{5}']).sort('date', descending=False)
# merged_df

In [28]:
# merged_df.sort('code')[10:20]

In [29]:
def calculate_csp_factor(i):
    start_time = time.time()
    ##第一步，聚合k线
    df_k_line = aggregate_k_lines(df_price,i)
    print(f"aggregate_k_lines 耗时: {time.time() - start_time:.2f}秒")
    ## 第二步，打上标签
    df_k_line = calculate_price_indicators(df_k_line)
    ## 第三步，计算pattern的聚合指标
    

    ir = calculate_ir_metrics(df_k_line, lag_days, n_days, min_samples)
    merged_df = ir.join(
        df_k_line.select(['date','pattern','code']),
        on=["date", "pattern"],
        how="inner"
    ).select( "date", "code","ir")
    merged_df = merged_df.with_columns(
        pl.col('ir').rolling_mean(20, min_periods=5).over('code').alias(f'factor_{i}') 
    ).drop_nulls(subset=[f'factor_{i}']).sort('date', descending=False)


    return merged_df.select("date", "code", f'factor_{i}')


In [30]:
def calculate_new_factor(i):
    start_time = time.time()
    ##第一步，聚合k线
    df_k_line = aggregate_k_lines(df_price,i)
    print(f"aggregate_k_lines 耗时: {time.time() - start_time:.2f}秒")
    ## 第二步，打上标签
    df_k_line = calculate_price_indicators(df_k_line)
    ## 第三步，计算pattern的聚合指标
    

    ir = calculate_ir_metrics(df_k_line, 0, n_days, min_samples)
    merged_df = ir.join(
        df_k_line.select(['date','pattern','code']),
        on=["date", "pattern"],
        how="inner"
    ).select( "date", "code","ir")
    merged_df = merged_df.with_columns(
        pl.col('ir').rolling_mean(20, min_periods=5).over('code').alias(f'factor_{i}') 
    ).drop_nulls(subset=[f'factor_{i}']).sort('date', descending=False)


    return merged_df.select("date", "code", f'factor_{i}')


In [31]:
# null_counts = merged_df.select([pl.col(col).is_null().sum().alias(col) for col in merged_df.columns])

# null_counts

In [32]:
# df.join(
#     df_k_line.select(['date','pattern','code']),
#     on=["date", "pattern"],
#     how="outer"
# )

## 批量计算

In [33]:
# 计算各周期的因子并合并
df = pl.DataFrame(df_price[['date','code']].unique())


for i in range(num1, num2):
    start_time = time.time()
    df_factor = calculate_csp_factor(i)
    if df.is_empty():
        df = df_factor
    else:
        df = df.join(df_factor, on=["date", "code"], how="left")
    # print(df)
    print(f"Completed i={i},耗时: {time.time() - start_time:.2f}秒")
df

aggregate_k_lines 耗时: 1.08秒
Completed i=1,耗时: 35.15秒
aggregate_k_lines 耗时: 1.39秒
Completed i=2,耗时: 31.43秒
aggregate_k_lines 耗时: 1.24秒
Completed i=3,耗时: 29.63秒
aggregate_k_lines 耗时: 1.04秒
Completed i=4,耗时: 29.58秒
aggregate_k_lines 耗时: 1.00秒
Completed i=5,耗时: 28.25秒
aggregate_k_lines 耗时: 1.05秒
Completed i=6,耗时: 28.49秒
aggregate_k_lines 耗时: 1.03秒
Completed i=7,耗时: 28.03秒
aggregate_k_lines 耗时: 1.04秒
Completed i=8,耗时: 27.23秒
aggregate_k_lines 耗时: 0.94秒
Completed i=9,耗时: 25.78秒
aggregate_k_lines 耗时: 0.99秒
Completed i=10,耗时: 25.51秒


date,code,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,factor_9,factor_10
str,cat,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2018-01-29""","""300097.SZ""",0.217092,0.217499,0.234387,0.246604,0.268156,0.257111,0.222683,0.212859,0.177128,0.163421
"""2022-11-02""","""688358.SH""",0.064537,-0.031687,-0.013892,-0.001166,-0.041755,-0.027034,-0.00591,0.01759,0.012802,0.016141
"""2024-03-28""","""873132.BJ""",-0.006153,0.033767,0.017809,0.01894,0.02231,0.013366,0.019009,0.025862,0.023769,0.013928
"""2021-11-15""","""000978.SZ""",0.167368,0.216943,0.202248,0.194482,0.177087,0.162011,0.171817,0.157894,0.195466,0.160468
"""2019-05-23""","""300448.SZ""",0.221652,0.162924,0.16568,0.192428,0.210846,0.171733,0.167867,0.153652,0.169834,0.158218
…,…,…,…,…,…,…,…,…,…,…,…
"""2024-12-05""","""601068.SH""",0.032325,0.074748,0.053557,0.014765,0.066498,0.018949,0.033021,0.031588,0.051241,0.036646
"""2016-02-17""","""002627.SZ""",0.316595,0.223323,0.204957,0.190711,0.214378,0.214463,0.190005,0.209521,0.18289,0.218275
"""2024-01-22""","""601096.SH""",null,null,null,null,null,null,null,null,null,null


In [34]:
null_counts = df.select([pl.col(col).is_null().sum().alias(col) for col in df.columns])

null_counts

date,code,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,factor_9,factor_10
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,159873,136713,135481,134984,134241,133741,133536,133504,133051,132878


In [35]:
df_filled = df.fill_null(0)
# df_filled = df.fill_nan(0)
df_filled

date,code,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,factor_9,factor_10
str,cat,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""2018-01-29""","""300097.SZ""",0.217092,0.217499,0.234387,0.246604,0.268156,0.257111,0.222683,0.212859,0.177128,0.163421
"""2022-11-02""","""688358.SH""",0.064537,-0.031687,-0.013892,-0.001166,-0.041755,-0.027034,-0.00591,0.01759,0.012802,0.016141
"""2024-03-28""","""873132.BJ""",-0.006153,0.033767,0.017809,0.01894,0.02231,0.013366,0.019009,0.025862,0.023769,0.013928
"""2021-11-15""","""000978.SZ""",0.167368,0.216943,0.202248,0.194482,0.177087,0.162011,0.171817,0.157894,0.195466,0.160468
"""2019-05-23""","""300448.SZ""",0.221652,0.162924,0.16568,0.192428,0.210846,0.171733,0.167867,0.153652,0.169834,0.158218
…,…,…,…,…,…,…,…,…,…,…,…
"""2024-12-05""","""601068.SH""",0.032325,0.074748,0.053557,0.014765,0.066498,0.018949,0.033021,0.031588,0.051241,0.036646
"""2016-02-17""","""002627.SZ""",0.316595,0.223323,0.204957,0.190711,0.214378,0.214463,0.190005,0.209521,0.18289,0.218275
"""2024-01-22""","""601096.SH""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
null_counts = df_filled.select([pl.col(col).is_null().sum().alias(col) for col in df_filled.columns])

null_counts

date,code,factor_1,factor_2,factor_3,factor_4,factor_5,factor_6,factor_7,factor_8,factor_9,factor_10
u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:

a = df_filled.with_columns(
    factor = (pl.col(f'factor_1')+pl.col(f'factor_2')+pl.col(f'factor_3')+pl.col(f'factor_4')+pl.col(f'factor_5')+pl.col(f'factor_6')+pl.col(f'factor_7')+pl.col(f'factor_8')+pl.col(f'factor_9')+pl.col(f'factor_10'))/10
).select(['date','code','factor']).filter(pl.col('factor') != 0).sort('date', descending=False)
a

date,code,factor
str,cat,f64
"""2010-02-04""","""300019.SZ""",-0.078637
"""2010-02-04""","""002062.SZ""",-0.001427
"""2010-02-04""","""000582.SZ""",0.025962
"""2010-02-04""","""000520.SZ""",0.139519
"""2010-02-04""","""000421.SZ""",0.19905
…,…,…
"""2025-02-13""","""002081.SZ""",0.627637
"""2025-02-13""","""600796.SH""",0.144479
"""2025-02-13""","""603058.SH""",0.258738


In [38]:
null_counts = a.select([pl.col(col).is_null().sum().alias(col) for col in a.columns])

null_counts

date,code,factor
u32,u32,u32
0,0,0


In [45]:
a.filter((pl.col('date')=='2025-02-13')&(pl.col('code')=='000988.SZ'))

date,code,factor
str,cat,f64
"""2025-02-13""","""000988.SZ""",-0.090852


In [46]:
print(0.922901/0.090852)

10.158290406375203


In [39]:
# a.write_csv('csp_composite.csv')

In [40]:
# df.write_csv('csp_all.csv')

In [41]:
# df.select(['date','code','factor_5']).sort(['date','code']).write_csv('csp_5.csv')

In [ ]:
a.f